El modelo a resolver en Pyomo es:


\begin{aligned}
&	\text{maximizar} & Ben=\sum_{i=1}^{Pan}P_i\cdot P^{venta}_i-\sum_{j=1}^{Mat\_primas}F_j\cdot P^{compra}_j \\
& \text{sujeto a:} \\
&  F_H = 0.7*P_1+0.6*P_2+0.8*P_3+0.9*P_4 &\text{Balance Har}\\
&   F_L = 0.01*P_1+0.05*P_2+0.07*P_3+0.1*P_4 &\text{Balance Lev}\\
&   F_E = 4*P_1+3.5*P_2+3*P_3+4.5*P_4 &\text{Balance Elec}\\
&  F_j \leq Stock_j &\text{(Máximo cantidad de materias primas)}\\
&  P_i \geq Demanda_i &\text{(Mínima demanda de productos)}\\
&  F_j, P_i \geq 0 
\end{aligned}



Importamos librerías y creamos el modelo:

In [1]:
from pyomo.environ import *
model = ConcreteModel()

Las únicas variables que tenemos en el modelo son la cantidad de materias primas utilizadas ($F_j$) y de productos fabricada ($P_i$):

In [2]:
model.FH =Var(within = NonNegativeReals)
model.FL =Var(within = NonNegativeReals)
model.FE =Var(within = NonNegativeReals)
model.P1 = Var(within = NonNegativeReals)
model.P2 = Var(within = NonNegativeReals)
model.P3 = Var(within = NonNegativeReals)
model.P4 = Var(within = NonNegativeReals)

La función objetivo tiene dos contribuciones: venta de productos-consumo de materias primas

In [3]:
model.Ben = Objective(expr = 5*model.P1+6*model.P2+4*model.P3+8*model.P4-model.FH*0.8-model.FL*2.5-model.FE*0.2, sense = maximize)

Creamos las restricciones. Para ello, usamos *Constraint* ya que son expresiones sencillas 

In [4]:
model.BalH = Constraint(expr = model.FH== model.P1*0.7+model.P2*0.6+model.P3*0.8+model.P4*0.9)

model.BalL = Constraint(expr = model.FL== model.P1*0.01+model.P2*0.05+model.P3*0.07+model.P4*0.1)

model.BalE = Constraint(expr = model.FE== model.P1*4+model.P2*3.5+model.P3*3+model.P4*4.5)

model.maxH = Constraint(expr = model.FH <= 2000)
model.maxL = Constraint(expr = model.FL <= 10)
model.maxE = Constraint(expr = model.FE <= 2400)

Por último, resolvemos el modelo:

In [5]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
model.rc = Suffix(direction=Suffix.IMPORT_EXPORT)

results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

print("Restr \t \t valor \t \t lslack\t \t uslack \t dual")
for c in [model.BalH, model.BalL, model.BalE, model.maxH, model.maxL, model.maxE ]:
    print(c , " \t \t{0:4.0f} \t \t{1:4.1f} \t \t{2:4.1f} \t \t {3:4.1f} \t \t".format(c(), c.lslack(), c.uslack(), model.dual[c]))

7 Var Declarations
    FE : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2400.0 :  None : False : False : NonNegativeReals
    FH : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 418.787878787879 :  None : False : False : NonNegativeReals
    FL : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  10.0 :  None : False : False : NonNegativeReals
    P1 : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 515.151515151515 :  None : False : False : NonNegativeReals
    P2 : Size=1, Index=None
        Key  : Lower : Value           : Upper : Fixed : Stale : Domain
        None :     0 : 96.969696969697 :  None : False : False : NonNegativeReals
    P3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None : 

In [6]:
model.minP1 = Constraint(expr = model.P1 >= 80)
model.minP2 = Constraint(expr = model.P2 >= 60)
model.minP3 = Constraint(expr = model.P3 >= 25)
model.minP4 = Constraint(expr = model.P4 >= 25)

In [7]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
model.rc = Suffix(direction=Suffix.IMPORT_EXPORT)

results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

print("Restr \t \t valor \t \t lslack\t \t uslack \t dual")
for c in [model.BalH, model.BalL, model.BalE, model.maxH, model.maxL, model.maxE ]:
    print(c , " \t \t{0:4.0f} \t \t{1:4.1f} \t \t{2:4.1f} \t \t {3:4.1f} \t \t".format(c(), c.lslack(), c.uslack(), model.dual[c]))

    'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
    'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
7 Var Declarations
    FE : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1497.5 :  None : False : False : NonNegativeReals
    FH : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 271.0 :  None : False : False : NonNegativeReals
    FL : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  10.0 :  None : Fals